In [ ]:
# Importing required libraries..
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import pandas as pd

In [ ]:
# Creating chrome web driver..
browser = webdriver.Chrome('/Users/iz13m/Code/Linkedin/chromedriver')
browser.set_window_position(900, 0)
browser.set_window_size(500, 800)

In [ ]:
#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = browser.find_element_by_id('username')
elementID.send_keys('*********@gmail.com')

elementID = browser.find_element_by_id('password')
elementID.send_keys('**********')
elementID.submit()

In [ ]:
# Job search page
URL = 'https://www.linkedin.com/jobs/search/?geoId=106155005&location=Egypt&sortBy=DD'

In [ ]:
def paging(url):
    # This function returns pages urls
    # It's defaulted to return 199 pages.. you can edit the number
    browser.get(url)
    time.sleep(1)
    lst = []
    for i in range(200):
        link = url +"&start={}".format(7*i+1)
        lst.append(link)
    print('Pages Created Successfully .. ')
    return lst

In [ ]:
def get_jobs(url):
    # This function return jobs urls in given jobs search page.. 
    browser.get(url)
    time.sleep(2)
    lst = []
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    job_containers = soup.find_all('div', class_ = 'full-width artdeco-entity-lockup__title ember-view')
    link = str(job_containers[0])
    jobs=[]
    try:
        for i in range(len(job_containers)):
            link = str(job_containers[i])
            try:
                jobid = re.findall(r"jobs/view/\d\d\d\d\d\d\d\d\d\d",link)[0].split('/')[-1]
                jobs.append("https://www.linkedin.com/jobs/view/" + jobid)
                print(i,' success')
            except:
                print(i, "skipped")
    except:
        print("loop skipped..")
    return jobs

In [ ]:
# This part starts the process of creating pages url and then adding them to a list
pages = paging(URL)
all_links = []
for page in pages:
    links = get_jobs(page)
    all_links.extend(links)

In [ ]:
# Checking for the count of urls
len(all_links)

In [ ]:
def scrapper(url):
    # This function is responsible for scrapping job information of a given job url
    browser.get(url)
    time.sleep(2)
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    values={}
    
    job_title = soup.find('h1', class_ = 't-24 t-bold').text
    try:
        company_name = soup.find('span', class_ = 'jobs-unified-top-card__subtitle-primary-grouping mr2 t-black').span.text.strip()
    except:
        company_name=""
    try:
        location = soup.find('span', class_ = 'jobs-unified-top-card__bullet').text.strip()
    except:
        location=""
    try:
        date = soup.find('span', class_ = 'jobs-unified-top-card__posted-date').text.strip()
    except:
        date=""
    try:
        job_bar = soup.find(attrs={"type":"briefcase-icon"}).parent.parent.parent.next_sibling.next_sibling.text
    except:
        job_bar=""
    try:
        com_bar = soup.find(attrs={"type":"company-icon"}).parent.parent.parent.next_sibling.next_sibling.text
    except:
        com_bar = ""
    try:
        job_type = job_bar.split("·")[0]
    except:
        job_type = ""
    try:
        job_level = job_bar.split("·")[1]
    except:
        job_level=""
    try:
        emp_count = com_bar.split("·")[0] 
    except:
        emp_count=""
    try:
        industry = com_bar.split("·")[1]
    except:
        industry = ""
    try:
        salary = soup.find('h2', class_= 't-16 pt4 ph5').text.strip()
    except:
        salary=""
        
    description = " ".join([text for text in soup.find(id="job-details").stripped_strings])
        
    values = {"Title":job_title,"Company Name": company_name, "Location":location, "Date":date,
              "Job Type":job_type, "Seneiority Level":job_level, "Company Size": emp_count,
             "Industry": industry, "About The Job": description, "Pay Range": salary, "Job URL":url}
    return values

In [ ]:
# This part feeds the scrapper function jobs urls from a staring index
# Starting index is used as the code may break during scrapping ..
# .. due to server errors from linkedin or any other internet related problems ..
# .. which enables us to continue from where we stopped.
df = []
start_pos = 369
for i in range(start_pos,len(all_links)):
    dic = scrapper(all_links[i])
    df.append(dic)
    print(i," Success")
    time.sleep(1)

In [ ]:
# Checking for the count of jobs..
len(df)

In [ ]:
# Converting data into pandas dataframe ..
df_ = pd.DataFrame(df)
df_.shape

In [ ]:
# Exporting data into a CSV file
df_.to_csv("Linkedin.csv")